In [94]:
import tkinter as tk
from tkinter import *
from tkinter.ttk import *
import heapq
from PIL import Image, ImageTk

class Node:
    def __init__(self, level, cost, assigned_jobs, total_cost, bound):
        self.level = level
        self.cost = cost
        self.assigned_jobs = assigned_jobs
        self.total_cost = total_cost
        self.bound = bound

    def __lt__(self, other):
        return self.bound < other.bound

def calc_bound(node, matrix):
    bound = node.total_cost
    for i in range(node.level, len(matrix)):
        if i not in node.assigned_jobs:
            min_cost = float('inf')
            for j in range(1, len(matrix[i])):
                if j not in node.assigned_jobs and matrix[i][j] < min_cost:
                    min_cost = matrix[i][j]
            bound += min_cost
    return bound

def branch_and_bound(matrix):
    n = len(matrix)
    priority_queue = []
    root = Node(0, 0, [], 0, 0)
    root.bound = calc_bound(root, matrix)
    heapq.heappush(priority_queue, root)

    min_cost = float('inf')
    best_assignment = []

    while priority_queue:
        node = heapq.heappop(priority_queue)

        if node.bound < min_cost:
            if node.level == n:
                min_cost = node.total_cost
                best_assignment = node.assigned_jobs[:]
            else:
                for i in range(n):
                    if i not in node.assigned_jobs:
                        new_cost = node.total_cost + matrix[node.level][i]
                        new_bound = new_cost + calc_bound(Node(node.level + 1, 0,
                                            node.assigned_jobs + [i],
                                            new_cost, 0), matrix)

                        if new_bound < min_cost:
                            heapq.heappush(priority_queue, Node(node.level + 1, 0,
                                                node.assigned_jobs + [i],
                                                new_cost, new_bound))

    return min_cost, best_assignment

def show_entry_fields(entries, language="English"):
    matrix = [[0]*n for _ in range(n)]
    c = 0
    for i in range(n):
        for j in range(n):
            matrix[i][j] = int(entries[j+c].get())
        c += n
    min_cost, best_assignment = branch_and_bound(matrix)
    update_results(min_cost, best_assignment, language)

def update_results(min_cost, best_assignment, language="English"):
    if language == 'Persian':
        result_label.config(text="کمترین هزینه: {}\nتخصیص: {}".format(min_cost, best_assignment))
    else:
        result_label.config(text="Minimum cost: {}\nAllocation: {}".format(min_cost, best_assignment))

def open_matrix_window(language_var):
    global matrix_frame
    matrix_window = tk.Toplevel(root)

    matrix_window.title("Matrix Input")
    
    global n
    n = int(n_entry.get())

    matrix_frame = tk.Frame(matrix_window, bg='#FFE1E1')  # Set a light salmon background color
    matrix_frame.pack(padx=5, pady=5)

    entries = [tk.Entry(matrix_frame, bg='#FFE1E1') for _ in range(n*n)]  # Set a light salmon background color
    positions = [(i, j) for i in range(n) for j in range(n)]
    for index, entry in enumerate(entries):
        row, column = positions[index]
        entry.grid(row=row, column=column, padx=1, pady=1)

    language_label = tk.Label(matrix_window, text="Select Language:")
    language_label.pack()

    language_options = ['Persian', 'English']
    language_var.set(language_options[0])

    language_dropdown = OptionMenu(matrix_window, language_var, *language_options)
    language_dropdown.pack()

    Button(matrix_window, text='Quit', command=matrix_window.destroy).pack(side=LEFT)
    Button(matrix_window, text='Execute', command=lambda: show_entry_fields(entries, language_var.get())).pack(side=RIGHT)

    global result_label
    result_label = tk.Label(matrix_window, text="Results will be displayed here")
    result_label.pack()
    
root = tk.Tk()
bg = '#6B0504'
root.configure(bg=bg)

root.title("Branch&Bound")
root.geometry('300x150')
root.title("nxn Matrix Input")

n_label = tk.Label(root, text="Enter the size 'n' of the matrix:", fg="#FBFFFE", bg=bg,
                   font=("Ink Free", 14))
n_label.pack(padx=10, pady=10)


n_entry = tk.Entry(root)
n_entry.pack(padx=10, pady=10)

submit_button = tk.Button(root, text="Create Matrix", command=lambda: open_matrix_window(language_var))
submit_button.pack(padx=10, pady=20)

language_var = StringVar()

root.mainloop()